In [1]:
#EDA

In [2]:
import torch

In [3]:
#!pip install pysentimiento transformers datasets accelerate evaluate

In [4]:
import datasets
import evaluate

2024-03-03 17:22:12.627991: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 17:22:12.628018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 17:22:12.628975: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 17:22:12.634900: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 17:22:13.268332: W tensorflow/compiler/tf2

In [5]:
import numpy as np
from datasets import load_dataset

In [6]:
import ipywidgets as widgets

In [7]:
#Helper Functions

In [8]:
# label to name
def label2name(x):
    if x == 0:
        return "Negative"
    if x == 1:
        return "Neutral"
    if x == 2:
        return "Positive"

In [9]:

f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    
    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="macro"))
    return results

In [10]:
#Read Data


In [11]:
data_files = {"train": "data/train.csv", "validation": "data/val.csv", "test": "data/test.csv"}
ds = load_dataset("csv", data_files=data_files)

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 8877
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 1097
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 987
    })
})

In [13]:
ds["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_name': Value(dtype='string', id=None),
 'tokenized_text': Value(dtype='string', id=None),
 'sent_token_length': Value(dtype='int64', id=None),
 'sent_bert_token_length': Value(dtype='int64', id=None),
 'char_count': Value(dtype='int64', id=None),
 'Character Count': Value(dtype='int64', id=None)}

In [14]:
ds["test"]["label"][:10]

[2, 1, 1, 1, 2, 1, 0, 1, 2, 1]

In [15]:
df = ds['train'].to_pandas()
df.head()

,text,label,label_name,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
0,Que ahora se pelean por tener la papaya peladi...,0,Negative,Que ahora se pelean por tener la papaya peladi...,16,19,79,79
1,"Beto, chicote también para los chivitos 😂",0,Negative,"Beto, chicote también para los chivitos 😂",7,11,41,41
2,Decidamos todo! = Ing y 2k 👍= Vago y 7k,1,Neutral,Decidamos todo! = Ing y 2k 👍= Vago y 7k,10,14,40,40
3,Faltaron los ricos chupetes de pascual saco😋,2,Positive,Faltaron los ricos chupetes de pascual saco😋,7,9,44,44
4,Tú hermana debería estar presa y tal ves ahí p...,0,Negative,Tú hermana debería estar presa y tal ves ahí p...,94,104,513,513


In [16]:
# encode label and mapping label name
#df["label"] = df["label"].apply(lambda x: label_encode(x))
df["label_name"] = df["label"].apply(lambda x: label2name(x))

In [17]:
# clean text, lowercase and remove punk
#df["text"] = df["text"].apply(lambda x: remove_punct(clean(remove_emoji(x).lower())[0][0]))

In [18]:
df.head()

,text,label,label_name,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
0,Que ahora se pelean por tener la papaya peladi...,0,Negative,Que ahora se pelean por tener la papaya peladi...,16,19,79,79
1,"Beto, chicote también para los chivitos 😂",0,Negative,"Beto, chicote también para los chivitos 😂",7,11,41,41
2,Decidamos todo! = Ing y 2k 👍= Vago y 7k,1,Neutral,Decidamos todo! = Ing y 2k 👍= Vago y 7k,10,14,40,40
3,Faltaron los ricos chupetes de pascual saco😋,2,Positive,Faltaron los ricos chupetes de pascual saco😋,7,9,44,44
4,Tú hermana debería estar presa y tal ves ahí p...,0,Negative,Tú hermana debería estar presa y tal ves ahí p...,94,104,513,513


In [19]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "pysentimiento/robertuito-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# columns = ['input_ids', 'attention_mask', 'label']
# ds.set_format(type='torch', columns=columns)
# ds

In [21]:
from pysentimiento.preprocessing import preprocess_tweet
preprocessed_ds = ds.map(lambda ex: {"text": preprocess_tweet(ex["text"], lang="es")})

In [22]:
tokenized_ds = preprocessed_ds.map(
    lambda batch: tokenizer(
        batch["text"], padding=True, truncation=True
        ),
    batched=True, batch_size=32
)

Map:   0%|          | 0/1097 [00:00<?, ? examples/s]

In [23]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8877
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1097
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 987
    })
})

In [24]:
#tokenized_ds['train']['input_ids'][0]

In [25]:
#tokenized_ds['train']['attention_mask'][0]

In [26]:
tokenized_ds['train']['text'][0]

'Que ahora se pelean por tener la papaya peladita o. L. Amas grande y usada  emoji cara revolviéndose de la risa emoji  emoji cara revolviéndose de la risa emoji  emoji cara revolviéndose de la risa emoji  emoji berenjena emoji'

In [27]:
#!pip install ipdb

In [28]:
#import torch
#device = "cuda" if torch.cuda.is_available() else "cpu"
#model.cuda()

In [29]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

training_args = TrainingArguments(
    per_device_train_batch_size=32,
    output_dir="test_trainer",
    do_eval=True,
    evaluation_strategy="epoch",
    num_train_epochs=7,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [30]:
trainer.train()

{'eval_loss': 0.6338252425193787, 'eval_f1': 0.7116585641659948, 'eval_recall': 0.7214898508756242, 'eval_runtime': 5.599, 'eval_samples_per_second': 195.929, 'eval_steps_per_second': 24.647, 'epoch': 1.0}
{'loss': 0.5347, 'learning_rate': 3.7153134635149026e-05, 'epoch': 1.8}
{'eval_loss': 0.6584495902061462, 'eval_f1': 0.7322100529541052, 'eval_recall': 0.7347706673911946, 'eval_runtime': 5.4147, 'eval_samples_per_second': 202.596, 'eval_steps_per_second': 25.486, 'epoch': 2.0}
{'eval_loss': 0.9159330725669861, 'eval_f1': 0.7302613134678322, 'eval_recall': 0.7339941355646963, 'eval_runtime': 5.5602, 'eval_samples_per_second': 197.295, 'eval_steps_per_second': 24.819, 'epoch': 3.0}
{'loss': 0.1759, 'learning_rate': 2.430626927029805e-05, 'epoch': 3.6}
{'eval_loss': 1.3203222751617432, 'eval_f1': 0.7275967282275997, 'eval_recall': 0.732059634273844, 'eval_runtime': 5.4559, 'eval_samples_per_second': 201.066, 'eval_steps_per_second': 25.294, 'epoch': 4.0}
{'eval_loss': 1.561045885086059

TrainOutput(global_step=1946, training_loss=0.19486353723265284, metrics={'train_runtime': 970.0053, 'train_samples_per_second': 64.06, 'train_steps_per_second': 2.006, 'train_loss': 0.19486353723265284, 'epoch': 7.0})

In [31]:
#Test on validation set

In [32]:
trainer.evaluate(tokenized_ds["test"])

{'eval_loss': 1.6248438358306885, 'eval_f1': 0.7321018216532135, 'eval_recall': 0.7315214354746157, 'eval_runtime': 4.9101, 'eval_samples_per_second': 201.015, 'eval_steps_per_second': 25.254, 'epoch': 7.0}


{'eval_loss': 1.6248438358306885,
 'eval_f1': 0.7321018216532135,
 'eval_recall': 0.7315214354746157,
 'eval_runtime': 4.9101,
 'eval_samples_per_second': 201.015,
 'eval_steps_per_second': 25.254,
 'epoch': 7.0}

In [33]:
ds['test']['text'][0]

'😂😂🤣🤣🤣...UNA CHAMBA DE LA CHACÔNDESUDREMA!!!😂😂🤣🤣🤣🍻🍻🍻'

In [34]:
#model.load_state_dict(torch.load(f'./_BERT_epoch_3.model', map_location=torch.device('cpu')))

In [35]:
#Error Analisys

In [36]:
val_df = ds['validation'].to_pandas()
#val_df

In [37]:
from tqdm.notebook import tqdm

In [38]:
# step by step predictions on dataframe
# We do this to view predictions in the pandas dataframe and easily filter them and perform error analysis.
pred_final = []

# for i, row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
#     predictions = []

#     text = row["text"]
#     encoded_data_test_single = tokenizer.batch_encode_plus([text], 
#     add_special_tokens=config.add_special_tokens, 
#     return_attention_mask=config.return_attention_mask, 
#     pad_to_max_length=config.pad_to_max_length, 
#     max_length=config.seq_length,
#     return_tensors=config.return_tensors
#     )
#     input_ids_test = encoded_data_test_single['input_ids']
#     attention_masks_test = encoded_data_test_single['attention_mask']

    
#     inputs = {'input_ids':      input_ids_test.to(device),
#               'attention_mask':attention_masks_test.to(device),
#              }

#     with torch.no_grad():        
#         outputs = model(**inputs)
    
#     logits = outputs[0]
#     logits = logits.detach().cpu().numpy()
#     predictions.append(logits)
#     predictions = np.concatenate(predictions, axis=0)
#     pred_final.append(np.argmax(predictions, axis=1).flatten()[0])

In [39]:
df.sample(n = 30)

,text,label,label_name,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
7973,Cuando me fui a marcahuasi casi muero de hipot...,0,Negative,Cuando me fui a marcahuasi casi muero de hipot...,49,56,248,248
5159,Florcita Bermudes Paucar vieja acuérdate los ...,1,Neutral,Florcita Bermudes Paucar vieja acuérdate los ...,24,38,135,135
3206,Solo falta su arroz ese plato 😀,0,Negative,Solo falta su arroz ese plato 😀,7,7,31,31
2917,Por qué tan lindo tamare lucianooo??? 👑😥💔 Jja...,1,Neutral,Por qué tan lindo tamare lucianooo??? 👑😥💔 Jja...,9,14,51,51
3626,"Hermoso tema ,grande Susan!! Gracias Eva, el v...",2,Positive,"Hermoso tema ,grande Susan!! Gracias Eva, el v...",13,20,85,85
3969,La policía anda pensando que eres un subversivo,1,Neutral,La policía anda pensando que eres un subversivo,8,10,47,47
3708,Me encantó claro tu rutina muy profesional👍,2,Positive,Me encantó claro tu rutina muy profesional👍,7,7,43,43
5171,Jjaja Yara que le laves la cara 🤪,0,Negative,Jjaja Yara que le laves la cara 🤪,8,9,33,33
7436,"🤣en buena hora que no le invitaron, la beto es...",0,Negative,"🤣en buena hora que no le invitaron, la beto es...",11,13,56,56
7910,Que te saque los piojos 😁,1,Neutral,Que te saque los piojos 😁,6,7,25,25


In [40]:
#Inference

In [41]:
#Evaluate the Model Qualitatively (Human Evaluation)

In [42]:
#Evaluate the Model Quantitatively (with F1 Metric)

In [43]:
texts = ds['test'][0:10]['text']
human_baseline_labels = ds['test'][0:10]['label']

In [44]:
texts

['😂😂🤣🤣🤣...UNA CHAMBA DE LA CHACÔNDESUDREMA!!!😂😂🤣🤣🤣🍻🍻🍻',
 'Maria Fernanda... ojalá y no me pegues tu adicción de ver series y deje de ir a tonear🤣',
 'Mi abuelita de 120 años también irá a botar 😂',
 'Alejandro un point a donde ir cuando estemos allá 😅',
 'Mi bello Huaraz 😍 Ayayayayy San Pancho !!!',
 'Y uno con síndrome de pájaro carpintero 🤤😁',
 'Dos cholos ebrios, gritaron: -Atajen ese globo',
 'Cristian Broncano Urbano  se parece a ti cuándo te rayas con algo y no paras j😂😂😂',
 'También abrazo a los árboles 🤗 y se siente bacán.  😊',
 'Kariii, siempre pensé que el cabello largo como estaba antes te queda genial. Te ves más chibola. No te lo vuelvas a cortar 😘']

In [45]:
human_baseline_labels

[2, 1, 1, 1, 2, 1, 0, 1, 2, 1]

In [46]:
#original_model_summaries = []
model_classifications = []

In [47]:
for _, text in enumerate(texts):
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to('cuda')
    logits = model(input_ids).logits
    probabilities = logits.softmax(dim=-1).tolist()[0]
    model_classifications.append(np.argmax(probabilities).flatten()[0])

In [48]:
model_classifications

[1, 1, 1, 1, 2, 1, 0, 1, 2, 2]